In [16]:
import wikipediaapi

wiki_wiki = wikipediaapi.Wikipedia('MyProjectName', 'en',
        extract_format=wikipediaapi.ExtractFormat.WIKI
)
main_wiki_word = "breakfast"
p_wiki = wiki_wiki.page(main_wiki_word)
text = p_wiki.text
print('text: ', text)

text:  Breakfast is the first meal of the day usually eaten in the morning. The word in English refers to breaking the fasting period of the previous night. Various "typical" or "traditional" breakfast menus exist, with food choices varying by regions and traditions worldwide.

History
In Old English, a regular morning meal was called morgenmete, and the word dinner, which originated from Gallo-Romance desjunare ("to break one's fast"), referred to a meal after fasting. Around the mid-13th century, that meaning of dinner faded away, and around the 15th century "breakfast" came into use in written English to describe a morning meal.

Ancient breakfast
Ancient Egypt
In Ancient Egypt, peasants ate a daily meal, most likely in the morning, consisting of soup, beer, bread, and onions before they left for work in the fields or work commanded by the pharaohs.
The traditional breakfast believed to have been cooked in ancient Egypt was fūl (made from fava beans, possibly the ancestor of today's

In [17]:
import re

def text_2_sen(text):
    # A basic sentence tokenizer
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    return sentences

def remove_text_from_start_end_marker(text, start_marker='(', end_marker=')'):
    # Remove parentheses and their content
    return re.sub(r'\{}.*?\{}'.format(re.escape(start_marker), re.escape(end_marker)), '', text).strip()

def convert_text2_sentences(text):

    sen = text_2_sen(text)
    # print('sen: ', sen)

    # --------------------------------
    # Add additional made-up sentences
    # --------------------------------
    # print('sen: ', sen)

    # --------------------------------
    # Clean the sentences
    # --------------------------------
    # Make sentences lowercase
    sen1 = [i.lower() for i in sen]

    # Remove parentheses and text in between parentheses
    sen2 = [remove_text_from_start_end_marker(i) for i in sen1]
    # print('sen2: ', sen2)

    # Split the sentences on a comma to make shorter sentences
    sen3 = []
    for i in sen2:
        temp = i.split(',')
        for j in temp:
            sen3.append(j.strip())
    # print('sen3: ', sen3)

    # Remove undesirable characters
    to_replace = ["!", ";", '\n', '</p>', '<a', 'id=', "href=", 'title=', 'class=', '</a>', '(', ')', '}', '{',
                  '</sup>', '<p>', '</b>', '<sup', '>', '<', '\\', '-']
    replace_with = ''

    sen4 = []
    for i in sen3:
        word_array = i.split()
        word_array_new = []
        for word in word_array:
            for to_replace_val in to_replace:
                word = word.replace(to_replace_val, replace_with)
            word_array_new.append(word)
        cleaned_sentence = ' '.join(word_array_new).strip()
        # Remove extra whitespaces
        cleaned_sentence = re.sub(r'\s+', ' ', cleaned_sentence)
        sen4.append(cleaned_sentence)
    # print('sen4: ', sen4)

    # --------------------------------
    # Remove sentences with less than 10 words.
    # --------------------------------
    sen5 = [i for i in sen4 if len(i.split()) > 10]
    # print('sen5: ', sen5)

    return sen5

# Example usage:
sentences = convert_text2_sentences(text)
print(sentences)

['breakfast is the first meal of the day usually eaten in the morning.', 'the word in english refers to breaking the fasting period of the previous night.', 'and around the 15th century "breakfast" came into use in written english to describe a morning meal.', 'and onions before they left for work in the fields or work commanded by the pharaohs.', 'the traditional breakfast believed to have been cooked in ancient egypt was fūl made from fava beans', 'the iliad notes this meal with regard to a laborweary woodsman eager for a light repast to start his day', "the opening prose of the 16th book of the odyssey mentions breakfast as the meal being prepared in the morning before attending to one's chores.", 'a meal called akratisma was typically consumed immediately after rising in the morning.', 'the earliest attested references on tagēnias are in the works of the 5th century bc poets cratinus and magnes.', '1st century latin poet martial said that ientaculum was eaten at 3:00 or 4:00 in the

In [18]:
#한 단어에 다음에 나오는 단어 추출
pattern = {}
for sentence in sentences:
    sentence,key = sentence.split(' '),None
    for word in sentence:
        if key == None: #전에 글자가 없으면 건너뛰기
            key = word
            continue
        if key not in pattern: pattern[key] = [{},0] #키가 없으면 추가
        if word in pattern[key][0]:
            pattern[key][0][word] += 1
        else:
            pattern[key][0][word] = 1
        pattern[key][1] += 1
        key = word #전 단어로 업데이트

pred = {}
for key in pattern.keys():
    arr,cnt = pattern[key]

    rarr = []
    for keyword,value in arr.items():
        rarr.append((keyword, value/cnt))
    pred[key] =rarr
    print(key, rarr)

for key, value in pred.items():
    print(key, value)

breakfast [('is', 0.125), ('believed', 0.03125), ('as', 0.03125), ('when', 0.03125), ('in', 0.0625), ('it', 0.03125), ('cuisine', 0.03125), ('usually', 0.03125), ('was', 0.0625), ('drinks', 0.03125), ('drink.', 0.03125), ('came', 0.03125), ('often', 0.03125), ('dish', 0.0625), ('cereal', 0.125), ('cereals', 0.0625), ('sandwich', 0.03125), ('beverage', 0.03125), ('high', 0.03125), ('q.v.', 0.03125), ('consumption', 0.03125), ('eating', 0.03125)]
is [('the', 0.08333333333333333), ('common', 0.08333333333333333), ('likely', 0.08333333333333333), ('known', 0.08333333333333333), ('that', 0.08333333333333333), ('said', 0.08333333333333333), ('estimated', 0.08333333333333333), ('a', 0.25), ('commonly', 0.08333333333333333), ('largely', 0.08333333333333333)]
the [('first', 0.07317073170731707), ('day', 0.012195121951219513), ('morning.', 0.04878048780487805), ('word', 0.012195121951219513), ('fasting', 0.012195121951219513), ('previous', 0.012195121951219513), ('15th', 0.012195121951219513), (

In [19]:
import random
import time

def tocken(now):
    #전처리
    if now not in pred: return True
    arr = pred[now]
    words = [item[0] for item in arr]
    persent = [item[1] for item in arr]
    #persent에 맞게 단어 랜덤으로 뽑기
    return random.choice(random.choices(words, weights=persent, k=3))

def token_continue(first_keyword):
    tk = tocken(first_keyword)
    if tk == True:
        print('No data.')
        return
    while tk != True:
        print(tk,end=' ')
        tk = tocken(tk)
        time.sleep(0.1)
    print()

print('----------'+main_wiki_word+'----------')
while True:
    print('📝사용자의 토큰')
    q = input()
    print(q)
    if q == 'stop_loop':
        print('📌종료되었습니다.')
        break
    print('✨예측된 토큰')
    token_continue(q)

----------breakfast----------
📝사용자의 토큰
breakfast
✨예측된 토큰
believed to memory was documented that it was then pounded and cooked in the modern day: a source is the bible cyclopædia et al. published in the origin of a dangerous meal: why you should ditch your morning by the first to eat sandwiches that mexican preschoolers consume 7% of the bible cyclopædia et al. it was typically eaten in rapidly available fruits 
📝사용자의 토큰
is
✨예측된 토큰
common in subjects of the same as the morning. 
📝사용자의 토큰
meal
✨예측된 토큰
of the first to the first groups q.v. 
📝사용자의 토큰
stop_loop
📌종료되었습니다.
